In [ ]:
pip install langgraph faiss-cpu langchain langchain-core langchain-google-genai langchain-community python-dotenv

In [274]:
import os
import pandas as pd
from typing import TypedDict, Annotated, Sequence
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# 0. SETUP API
# if "GOOGLE_API_KEY" not in os.environ:
    # os.environ["GOOGLE_API_KEY"] = "___"

# 1. DATA SETUP
# A. Secure ERP Data
data = {
    "product_id": ["GPU-X100", "GPU-X100", "CPU-Z50","GPU-X100"],
    "location": ["Taiwan", "Germany", "California","india"],
    "stock": [0, 50, 100,50], # GPU out of stock in Taiwan
    "supplier": ["TechGlobal", "EuroChips", "SiliconValley Inc","Swami Inc"]
}
df_inventory = pd.DataFrame(data)

# B. Unstructured News Data (Risks)
news_feed = [
    "URGENT: Super Typhoon 'Kuna' is hitting Taiwan. Ports closed.",
    "Strike Update: Germany logistics are operating normally.",
    "Wire Update:: Inida logistic is working normally"
]

In [275]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vector_store = FAISS.from_documents([Document(page_content=x) for x in news_feed], embeddings)

In [276]:
# 2. TOOLS (For the Agent)
@tool
def check_external_risks(location: str):
    """Searches news for risks in a specific location."""
    retriever = vector_store.as_retriever(search_kwargs={"k": 1})
    docs = retriever.invoke(location)
    return f"RISK REPORT: {docs[0].page_content}" if docs else "No risks found."

tools = [check_external_risks]

In [277]:
# 3. STATE
class SupplyChainState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    product_query: str
    erp_context: str # Stores the hard data string
    last_product: str
    last_location: str
    pending_decision: dict | None
    awaiting_human: bool
    alternatives: list
    decision_log: list

In [278]:
# 4. NODES

# NODE A: ERP Guard (Deterministic - No AI)
def erp_lookup_node(state: SupplyChainState):
    print("\n[Node: ERP Guard] querying database...")

    query = state["product_query"]
    matches = df_inventory[df_inventory['product_id'].str.contains(query, case=False)]

    if matches.empty:
        state["erp_context"] = "Product not found."
        state["last_product"] = ""
        state["last_location"] = ""
        state["alternatives"] = []
        return state

    # Identify primary (requested) location from user message
    user_msg = state["messages"][-1].content.lower()
    primary_location = None
    for loc in matches["location"].unique():
        if loc.lower() in user_msg:
            primary_location = loc
            break

    if not primary_location:
        primary_location = matches.iloc[0]["location"]

    # Build ERP context
    context = "FOUND INVENTORY:\n"
    for _, row in matches.iterrows():
        context += f"- Loc: {row['location']} | Stock: {row['stock']}\n"

    # ✅ DATA-DRIVEN alternative selection (THIS IS THE KEY)
    alternatives = (
        matches[
            (matches["stock"] > 0) &
            (matches["location"] != primary_location)
        ]["location"]
        .unique()
        .tolist()
    )

    state["erp_context"] = context
    state["last_product"] = matches.iloc[0]["product_id"]
    state["last_location"] = primary_location
    state["alternatives"] = alternatives

    return state

# NODE B: Agent (Probabilistic - AI)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key="replace-with-your-key").bind_tools(tools)

def agent_node(state: SupplyChainState):
    erp_data = state["erp_context"]

    sys_msg = SystemMessage(
        content=f"""
        ERP DATA (AUTHORITATIVE):
        {erp_data}
        
        TASK:
        - Identify risks
        - Identify alternative suppliers WITH STOCK
        - Propose options
        - Do NOT finalize decisions
        - Ask a question Like do they want to go ahead with alternative option that provided and take input
        """
    )

    response = llm.invoke([sys_msg] + state["messages"])
    state["messages"].append(response)

    # Proposal trigger (DATA-DRIVEN)
    if "FOUND INVENTORY" in erp_data and "Stock: 0" in erp_data:
        # alternatives = []
        for line in erp_data.splitlines():
            if "Stock:" in line and "0" not in line:
                loc = line.split("Loc:")[1].split("|")[0].strip()
                alternatives.append(loc)

            if state.get("alternatives"):
                state["pending_decision"] = {
                    "type": "ALTERNATIVE_LOCATION_AVAILABLE",
                    "product": state["last_product"],
                    "options": state["alternatives"]
                }

    return state

    
def approval_node(state: SupplyChainState):
    """
    This node PAUSES the graph and signals that human approval is required.
    """
    state["awaiting_human"] = True
    return state

In [ ]:
# 5. GRAPH BUILD
workflow = StateGraph(SupplyChainState)
workflow.add_node("erp_guard", erp_lookup_node)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", ToolNode(tools=tools))
workflow.add_node("approval", approval_node)

workflow.set_entry_point("erp_guard")
workflow.add_edge("erp_guard", "agent")

def router(state):
    last_msg = state["messages"][-1]

    if last_msg.tool_calls:
        return "tools"

    if state.get("pending_decision"):
        return "approval"

    return "end"

workflow.add_conditional_edges(
    "agent",
    router,
    {
        "tools": "tools",
        "approval": "approval",
        "end": END
    }
)

workflow.add_edge("tools", "agent")
workflow.add_edge("approval", END)

app = workflow.compile()

# 6. RUN
print("--- SCENARIO: Sourcing from Taiwan (Risky) ---")
inputs = {
    "product_query": "GPU",
    "messages": [
        HumanMessage(content="Can we source 50 GPU-X100s from Taiwan??")
    ],
    "last_product": "",
    "last_location": "",
    "alternatives": [],
    "pending_decision": None,
    "awaiting_human": False,
    "decision_log": []
}
for output in app.stream(inputs):
    pass

print(f"\n>>> FINAL ANSWER: {app.invoke(inputs)['messages'][-1].content[0]['text']}")
result = app.invoke(inputs)
if result.get("awaiting_human"):
    print("\n⏸ GRAPH PAUSED — HUMAN INPUT REQUIRED")

    decision = result["pending_decision"]
    options = [opt.lower() for opt in decision["options"]]

    print("Proposed alternative locations:", options)

    choice = input(
        f"Approve shipment from one of {options}? "
        "(type country name / yes / no): "
    ).strip().lower()

    approved_location = None

    # ✅ SAFE approval logic
    if choice == "yes":
        approved_location = options[0]
    elif choice in options:
        approved_location = choice

    # ---- LOG DECISION ----
    result["decision_log"].append({
        "decision": decision,
        "approved": approved_location is not None,
        "chosen_location": approved_location
    })

    if approved_location:
        result["messages"].append(
            SystemMessage(
                content=f"Human approved shipment from {approved_location.title()}."
            )
        )

        result["pending_decision"] = None
        result["awaiting_human"] = False

        # ▶ Resume graph
        
        print(f"Human approved shipment from {approved_location.title()}.")

    else:
        result["messages"].append(
            SystemMessage(content="Human rejected shipment. Order cancelled.")
        )

        result["pending_decision"] = None
        result["awaiting_human"] = False

        # ⛔ End workflow
        print("Human rejected shipment. Order cancelled.")

--- SCENARIO: Sourcing from Taiwan (Risky) ---

[Node: ERP Guard] querying database...

[Node: ERP Guard] querying database...

>>> FINAL ANSWER: Unfortunately, we cannot source 50 GPU-X100s from Taiwan.

**Risks identified:**
*   **No Stock:** Our inventory shows 0 units available in Taiwan.
*   **External Factors:** There is an urgent Super Typhoon 'Kuna' hitting Taiwan, which has resulted in port closures. This would prevent any shipments even if stock were available.

**Alternative Sourcing Options with Stock:**
*   **Germany:** We have 50 units in stock.
*   **India:** We have 50 units in stock.

Would you like to proceed with sourcing the 50 GPU-X100s from either Germany or India?

[Node: ERP Guard] querying database...

⏸ GRAPH PAUSED — HUMAN INPUT REQUIRED
Proposed alternative locations: ['germany', 'india']
